In [1]:
import requests
import json
import pandas as pd
from time import sleep

In [2]:
df = pd.read_csv('../output/codici_umbria_er_20241117.csv', dtype=str)
df_comuni = df[df['tipo'] == 'CM']
# Filtering for only Umbria
df_er = df_comuni[df_comuni['cod_regione'] == '10']

In [3]:
BASE_URL = 'https://eleapi.interno.gov.it/siel/PX/getprefeR/DE/20241117/TE/07/RE/'

df_er['url_aff'] = df_er.apply(
    lambda row: f"{BASE_URL}{row['cod_regione']}/PR/{row['cod_provincia_2']}/CM/{row['cod_comune']}", axis=1
)

df_er['url'] = df_er.apply(
    lambda row: f"https://elezioni.interno.gov.it/risultati/20241117/regionali/scrutini/italia/{row['cod_regione']}{row['cod_provincia']}{row['cod_comune']}", axis=1
)
df_er.sample(3)

/tmp/ipykernel_1804/3249733948.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_er['url_aff'] = df_er.apply(
/tmp/ipykernel_1804/3249733948.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_er['url'] = df_er.apply(


,cod,desc,tipo,tipo_comune,dt_agg,cod_ele,cod_regione,regione,cod_provincia,provincia,cod_provincia_2,cod_comune,url_aff,url
386,103040580350,NORCIA,CM,N,NaN,7,10,UMBRIA,304,TERNI,058,0350,https://eleapi.interno.gov.it/siel/PX/getprefe...,https://elezioni.interno.gov.it/risultati/2024...
422,103040800100,CASTEL VISCARDO,CM,N,NaN,7,10,UMBRIA,304,TERNI,080,0100,https://eleapi.interno.gov.it/siel/PX/getprefe...,https://elezioni.interno.gov.it/risultati/2024...
428,103040800160,LUGNANO IN TEVERINA,CM,N,NaN,7,10,UMBRIA,304,TERNI,080,0160,https://eleapi.interno.gov.it/siel/PX/getprefe...,https://elezioni.interno.gov.it/risultati/2024...


In [4]:
headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.6',
    'dnt': '1',
    'origin': 'https://elezioni.interno.gov.it',
    'priority': 'u=1, i',
    'referer': 'https://elezioni.interno.gov.it/',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sec-gpc': '1'
}

In [5]:
comuni_data = []

for idx, url in enumerate(df_er['url_aff']):
    print(f"\nProcessing record {idx + 1}")
    print(f"URL: {url}")

    response = requests.get(url, headers=headers)
    print(f"Response status code: {response.status_code}")
    
    data = response.json()

    comune = df_er.iloc[idx]['desc']
    provincia = df_er.iloc[idx]['provincia']
    regione = df_er.iloc[idx]['regione']
    url = df_er.iloc[idx]['url']

    print(f"Processing data for Comune: {comune}, Provincia: {provincia}")

    # Start with basic comune info
    comune_info = {
        'comune': comune,
        'provincia': provincia,
        'regione': regione,
        'url': url
    }
    
    # Add votes for each lista using lista description as column name
    if 'liste' in data:
        for lista in data['liste']:
            lista_desc = lista['desc']
            voti = lista.get('voti', 0)  # Use 0 if no votes found
            comune_info[lista_desc] = voti
            print(f"{lista_desc}: {voti} votes")

    if 'cand' in data:
        for cand in data['cand']:
            cand_desc = cand['nome'] + " " + cand['cogn']  # Use current candidate's name
            voti = cand['voti']
            comune_info[cand_desc] = voti
            print(f"{cand_desc}: {voti} votes")

    sezioni_scrutinate = data['liste'][0]['sez_perv']
    sezioni_tot = data['liste'][0]['sez_tot']

    # Optional: Verify consistency across lists
    if not all(lista.get('sez_perv') == sezioni_scrutinate for lista in data['liste']):
        print(f"Warning: Inconsistent scrutinized sections for {comune}")

    # Add to comune_info
    comune_info['sezioni_scrutinate'] = sezioni_scrutinate
    comune_info['sezioni_tot'] = sezioni_tot

    comuni_data.append(comune_info)
    
    print(f"Processed {comune}")
    sleep(0.1)

# Create DataFrame
print("\nCreating DataFrame...")
df_data = pd.DataFrame(comuni_data)

print("\nSample of processed data:")
print(df_data.head())


Processing record 1
URL: https://eleapi.interno.gov.it/siel/PX/getprefeR/DE/20241117/TE/07/RE/10/PR/058/CM/0010


Response status code: 200
Processing data for Comune: ASSISI, Provincia: TERNI
FRONTE DEL DISSENSO: 0 votes
QUINTO POLO PER L'ITALIA: 0 votes
INSIEME PER UN'UMBRIA RESISTENTE: 0 votes
PIU' ITALIA SOVRANA: 0 votes
LEGA: 0 votes
ALTERNATIVA POPOLARE: 0 votes
MOVIMENTO 5 STELLE: 0 votes
FORZA ITALIA: 0 votes
PARTITO DEMOCRATICO: 0 votes
UMBRIA PER LA SANITA' PUBBLICA: 0 votes
DEMOCRAZIA SOVRANA POPOLARE: 0 votes
FRATELLI D'ITALIA: 0 votes
TESEI PRESIDENTE: 0 votes
UNIONE DI CENTRO: 0 votes
NOI MODERATI - CIVICI PER L'UMBRIA: 0 votes
ALLEANZA VERDI E SINISTRA: 0 votes
CIVICI UMBRI: 0 votes
ALTERNATIVA PER L'UMBRIA: 0 votes
FORZA DEL POPOLO: 0 votes
ALTERNATIVA RIFORMISTA RIZZO PRESIDENTE: 0 votes
TRITTO PRESIDENTE UMANI INSIEME LIBERI: 0 votes
UMBRIA DOMANI PROIETTI PRESIDENTE: 0 votes
UMBRIA FUTURA - RIFORMISTI E CIVICI PROIETTI PRESIDENTE: 0 votes
FABIO RONCELLA: 0 votes
DANIELA DI MARCO: 0 votes
MARCELLO TETI: 0 votes
ENRICO SODACCI: 0 votes
ALESSANDRO VALLERIGNANI: 0 votes
MIRIAM SCARA

Response status code: 200
Processing data for Comune: BASTIA UMBRA, Provincia: TERNI
FRONTE DEL DISSENSO: 0 votes
QUINTO POLO PER L'ITALIA: 0 votes
INSIEME PER UN'UMBRIA RESISTENTE: 0 votes
PIU' ITALIA SOVRANA: 0 votes
LEGA: 0 votes
ALTERNATIVA POPOLARE: 0 votes
MOVIMENTO 5 STELLE: 0 votes
FORZA ITALIA: 0 votes
PARTITO DEMOCRATICO: 0 votes
UMBRIA PER LA SANITA' PUBBLICA: 0 votes
DEMOCRAZIA SOVRANA POPOLARE: 0 votes
FRATELLI D'ITALIA: 0 votes
TESEI PRESIDENTE: 0 votes
UNIONE DI CENTRO: 0 votes
NOI MODERATI - CIVICI PER L'UMBRIA: 0 votes
ALLEANZA VERDI E SINISTRA: 0 votes
CIVICI UMBRI: 0 votes
ALTERNATIVA PER L'UMBRIA: 0 votes
FORZA DEL POPOLO: 0 votes
ALTERNATIVA RIFORMISTA RIZZO PRESIDENTE: 0 votes
TRITTO PRESIDENTE UMANI INSIEME LIBERI: 0 votes
UMBRIA DOMANI PROIETTI PRESIDENTE: 0 votes
UMBRIA FUTURA - RIFORMISTI E CIVICI PROIETTI PRESIDENTE: 0 votes
FABIO RONCELLA: 0 votes
DANIELA DI MARCO: 0 votes
MARCELLO TETI: 0 votes
ENRICO SODACCI: 0 votes
ALESSANDRO VALLERIGNANI: 0 votes
MIRIAM

Response status code: 200
Processing data for Comune: BETTONA, Provincia: TERNI
FRONTE DEL DISSENSO: 0 votes
QUINTO POLO PER L'ITALIA: 0 votes
INSIEME PER UN'UMBRIA RESISTENTE: 0 votes
PIU' ITALIA SOVRANA: 0 votes
LEGA: 0 votes
ALTERNATIVA POPOLARE: 0 votes
MOVIMENTO 5 STELLE: 0 votes
FORZA ITALIA: 0 votes
PARTITO DEMOCRATICO: 0 votes
UMBRIA PER LA SANITA' PUBBLICA: 0 votes
DEMOCRAZIA SOVRANA POPOLARE: 0 votes
FRATELLI D'ITALIA: 0 votes
TESEI PRESIDENTE: 0 votes
UNIONE DI CENTRO: 0 votes
NOI MODERATI - CIVICI PER L'UMBRIA: 0 votes
ALLEANZA VERDI E SINISTRA: 0 votes
CIVICI UMBRI: 0 votes
ALTERNATIVA PER L'UMBRIA: 0 votes
FORZA DEL POPOLO: 0 votes
ALTERNATIVA RIFORMISTA RIZZO PRESIDENTE: 0 votes
TRITTO PRESIDENTE UMANI INSIEME LIBERI: 0 votes
UMBRIA DOMANI PROIETTI PRESIDENTE: 0 votes
UMBRIA FUTURA - RIFORMISTI E CIVICI PROIETTI PRESIDENTE: 0 votes
FABIO RONCELLA: 0 votes
DANIELA DI MARCO: 0 votes
MARCELLO TETI: 0 votes
ENRICO SODACCI: 0 votes
ALESSANDRO VALLERIGNANI: 0 votes
MIRIAM SCAR

Response status code: 200
Processing data for Comune: BEVAGNA, Provincia: TERNI
FRONTE DEL DISSENSO: 0 votes
QUINTO POLO PER L'ITALIA: 0 votes
INSIEME PER UN'UMBRIA RESISTENTE: 0 votes
PIU' ITALIA SOVRANA: 0 votes
LEGA: 0 votes
ALTERNATIVA POPOLARE: 0 votes
MOVIMENTO 5 STELLE: 0 votes
FORZA ITALIA: 0 votes
PARTITO DEMOCRATICO: 0 votes
UMBRIA PER LA SANITA' PUBBLICA: 0 votes
DEMOCRAZIA SOVRANA POPOLARE: 0 votes
FRATELLI D'ITALIA: 0 votes
TESEI PRESIDENTE: 0 votes
UNIONE DI CENTRO: 0 votes
NOI MODERATI - CIVICI PER L'UMBRIA: 0 votes
ALLEANZA VERDI E SINISTRA: 0 votes
CIVICI UMBRI: 0 votes
ALTERNATIVA PER L'UMBRIA: 0 votes
FORZA DEL POPOLO: 0 votes
ALTERNATIVA RIFORMISTA RIZZO PRESIDENTE: 0 votes
TRITTO PRESIDENTE UMANI INSIEME LIBERI: 0 votes
UMBRIA DOMANI PROIETTI PRESIDENTE: 0 votes
UMBRIA FUTURA - RIFORMISTI E CIVICI PROIETTI PRESIDENTE: 0 votes
FABIO RONCELLA: 0 votes
DANIELA DI MARCO: 0 votes
MARCELLO TETI: 0 votes
ENRICO SODACCI: 0 votes
ALESSANDRO VALLERIGNANI: 0 votes
MIRIAM SCAR

...

Response status code: 200
Processing data for Comune: PORANO, Provincia: TERNI
FRONTE DEL DISSENSO: 0 votes
QUINTO POLO PER L'ITALIA: 0 votes
INSIEME PER UN'UMBRIA RESISTENTE: 0 votes
PIU' ITALIA SOVRANA: 0 votes
LEGA: 0 votes
ALTERNATIVA POPOLARE: 0 votes
MOVIMENTO 5 STELLE: 0 votes
FORZA ITALIA: 0 votes
PARTITO DEMOCRATICO: 0 votes
UMBRIA PER LA SANITA' PUBBLICA: 0 votes
DEMOCRAZIA SOVRANA POPOLARE: 0 votes
FRATELLI D'ITALIA: 0 votes
TESEI PRESIDENTE: 0 votes
UNIONE DI CENTRO: 0 votes
NOI MODERATI - CIVICI PER L'UMBRIA: 0 votes
ALLEANZA VERDI E SINISTRA: 0 votes
CIVICI UMBRI: 0 votes
ALTERNATIVA PER L'UMBRIA: 0 votes
FORZA DEL POPOLO: 0 votes
ALTERNATIVA RIFORMISTA RIZZO PRESIDENTE: 0 votes
TRITTO PRESIDENTE UMANI INSIEME LIBERI: 0 votes
UMBRIA DOMANI PROIETTI PRESIDENTE: 0 votes
UMBRIA FUTURA - RIFORMISTI E CIVICI PROIETTI PRESIDENTE: 0 votes
FABIO RONCELLA: 0 votes
DANIELA DI MARCO: 0 votes
MARCELLO TETI: 0 votes
ENRICO SODACCI: 0 votes
ALESSANDRO VALLERIGNANI: 0 votes
MIRIAM SCARA

Response status code: 200
Processing data for Comune: SAN GEMINI, Provincia: TERNI
FRONTE DEL DISSENSO: 0 votes
QUINTO POLO PER L'ITALIA: 0 votes
INSIEME PER UN'UMBRIA RESISTENTE: 0 votes
PIU' ITALIA SOVRANA: 0 votes
LEGA: 0 votes
ALTERNATIVA POPOLARE: 0 votes
MOVIMENTO 5 STELLE: 0 votes
FORZA ITALIA: 0 votes
PARTITO DEMOCRATICO: 0 votes
UMBRIA PER LA SANITA' PUBBLICA: 0 votes
DEMOCRAZIA SOVRANA POPOLARE: 0 votes
FRATELLI D'ITALIA: 0 votes
TESEI PRESIDENTE: 0 votes
UNIONE DI CENTRO: 0 votes
NOI MODERATI - CIVICI PER L'UMBRIA: 0 votes
ALLEANZA VERDI E SINISTRA: 0 votes
CIVICI UMBRI: 0 votes
ALTERNATIVA PER L'UMBRIA: 0 votes
FORZA DEL POPOLO: 0 votes
ALTERNATIVA RIFORMISTA RIZZO PRESIDENTE: 0 votes
TRITTO PRESIDENTE UMANI INSIEME LIBERI: 0 votes
UMBRIA DOMANI PROIETTI PRESIDENTE: 0 votes
UMBRIA FUTURA - RIFORMISTI E CIVICI PROIETTI PRESIDENTE: 0 votes
FABIO RONCELLA: 0 votes
DANIELA DI MARCO: 0 votes
MARCELLO TETI: 0 votes
ENRICO SODACCI: 0 votes
ALESSANDRO VALLERIGNANI: 0 votes
MIRIAM S

Response status code: 200
Processing data for Comune: SAN VENANZO, Provincia: TERNI
FRONTE DEL DISSENSO: 0 votes
QUINTO POLO PER L'ITALIA: 0 votes
INSIEME PER UN'UMBRIA RESISTENTE: 0 votes
PIU' ITALIA SOVRANA: 0 votes
LEGA: 0 votes
ALTERNATIVA POPOLARE: 0 votes
MOVIMENTO 5 STELLE: 0 votes
FORZA ITALIA: 0 votes
PARTITO DEMOCRATICO: 0 votes
UMBRIA PER LA SANITA' PUBBLICA: 0 votes
DEMOCRAZIA SOVRANA POPOLARE: 0 votes
FRATELLI D'ITALIA: 0 votes
TESEI PRESIDENTE: 0 votes
UNIONE DI CENTRO: 0 votes
NOI MODERATI - CIVICI PER L'UMBRIA: 0 votes
ALLEANZA VERDI E SINISTRA: 0 votes
CIVICI UMBRI: 0 votes
ALTERNATIVA PER L'UMBRIA: 0 votes
FORZA DEL POPOLO: 0 votes
ALTERNATIVA RIFORMISTA RIZZO PRESIDENTE: 0 votes
TRITTO PRESIDENTE UMANI INSIEME LIBERI: 0 votes
UMBRIA DOMANI PROIETTI PRESIDENTE: 0 votes
UMBRIA FUTURA - RIFORMISTI E CIVICI PROIETTI PRESIDENTE: 0 votes
FABIO RONCELLA: 0 votes
DANIELA DI MARCO: 0 votes
MARCELLO TETI: 0 votes
ENRICO SODACCI: 0 votes
ALESSANDRO VALLERIGNANI: 0 votes
MIRIAM 

Response status code: 200
Processing data for Comune: STRONCONE, Provincia: TERNI
FRONTE DEL DISSENSO: 0 votes
QUINTO POLO PER L'ITALIA: 0 votes
INSIEME PER UN'UMBRIA RESISTENTE: 0 votes
PIU' ITALIA SOVRANA: 0 votes
LEGA: 0 votes
ALTERNATIVA POPOLARE: 0 votes
MOVIMENTO 5 STELLE: 0 votes
FORZA ITALIA: 0 votes
PARTITO DEMOCRATICO: 0 votes
UMBRIA PER LA SANITA' PUBBLICA: 0 votes
DEMOCRAZIA SOVRANA POPOLARE: 0 votes
FRATELLI D'ITALIA: 0 votes
TESEI PRESIDENTE: 0 votes
UNIONE DI CENTRO: 0 votes
NOI MODERATI - CIVICI PER L'UMBRIA: 0 votes
ALLEANZA VERDI E SINISTRA: 0 votes
CIVICI UMBRI: 0 votes
ALTERNATIVA PER L'UMBRIA: 0 votes
FORZA DEL POPOLO: 0 votes
ALTERNATIVA RIFORMISTA RIZZO PRESIDENTE: 0 votes
TRITTO PRESIDENTE UMANI INSIEME LIBERI: 0 votes
UMBRIA DOMANI PROIETTI PRESIDENTE: 0 votes
UMBRIA FUTURA - RIFORMISTI E CIVICI PROIETTI PRESIDENTE: 0 votes
FABIO RONCELLA: 0 votes
DANIELA DI MARCO: 0 votes
MARCELLO TETI: 0 votes
ENRICO SODACCI: 0 votes
ALESSANDRO VALLERIGNANI: 0 votes
MIRIAM SC

Response status code: 200
Processing data for Comune: TERNI, Provincia: TERNI
FRONTE DEL DISSENSO: 0 votes
QUINTO POLO PER L'ITALIA: 0 votes
INSIEME PER UN'UMBRIA RESISTENTE: 0 votes
PIU' ITALIA SOVRANA: 0 votes
LEGA: 0 votes
ALTERNATIVA POPOLARE: 0 votes
MOVIMENTO 5 STELLE: 0 votes
FORZA ITALIA: 0 votes
PARTITO DEMOCRATICO: 0 votes
UMBRIA PER LA SANITA' PUBBLICA: 0 votes
DEMOCRAZIA SOVRANA POPOLARE: 0 votes
FRATELLI D'ITALIA: 0 votes
TESEI PRESIDENTE: 0 votes
UNIONE DI CENTRO: 0 votes
NOI MODERATI - CIVICI PER L'UMBRIA: 0 votes
ALLEANZA VERDI E SINISTRA: 0 votes
CIVICI UMBRI: 0 votes
ALTERNATIVA PER L'UMBRIA: 0 votes
FORZA DEL POPOLO: 0 votes
ALTERNATIVA RIFORMISTA RIZZO PRESIDENTE: 0 votes
TRITTO PRESIDENTE UMANI INSIEME LIBERI: 0 votes
UMBRIA DOMANI PROIETTI PRESIDENTE: 0 votes
UMBRIA FUTURA - RIFORMISTI E CIVICI PROIETTI PRESIDENTE: 0 votes
FABIO RONCELLA: 0 votes
DANIELA DI MARCO: 0 votes
MARCELLO TETI: 0 votes
ENRICO SODACCI: 0 votes
ALESSANDRO VALLERIGNANI: 0 votes
MIRIAM SCARAB

In [6]:
data['cand'][0]['nome'] + " " + data['cand'][0]['cogn']

'FABIO RONCELLA'

In [7]:
df_data['comune'] = df_data['comune'].str.title()
df_data['provincia'] = df_data['provincia'].str.title()
df_data['regione'] = df_data['regione'].str.title()
df_data

,comune,provincia,regione,url,FRONTE DEL DISSENSO,QUINTO POLO PER L'ITALIA,INSIEME PER UN'UMBRIA RESISTENTE,PIU' ITALIA SOVRANA,LEGA,ALTERNATIVA POPOLARE,...,LORIANA GRASSELLI,GIACOMO LEONELLO LEONELLI,CATERINA LUCANGELI,MONICA MUZZI,DONATELLA PORZI,ROBERTO ROMANI,MAURIZIO SILVESTRELLI,TIZIANA TOMASSINI,sezioni_scrutinate,sezioni_tot
0,Assisi,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,32
1,Bastia Umbra,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,22
2,Bettona,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,Bevagna,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
4,Campello Sul Clitunno,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,Porano,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
88,San Gemini,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
89,San Venanzo,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,7
90,Stroncone,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6


In [8]:
df_data.to_csv('../output/risultati_um.csv', index=False, encoding='UTF-8-sig')